In [3]:
# import packages
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import gseapy as gp
import plotly.graph_objects as go
import plotly.express as px

In [4]:
# set working directory
work_dir = '/home/project/organoids'
repo_dir = f'{work_dir}/project_labubu'
data_dir = f'{work_dir}/transfered_from_galaxy'
results_path = f'{repo_dir}/results/dge_gsea'
# create results directory if does not exist
os.makedirs(results_path,exist_ok=True)

In [ ]:
# fill dataset metadata
# add dataset number and name
dat_met = {'ERP142256':{'name':'CRC tumoroid'},
          'ERP111852':{'name':'Lung bud tip'}}
# add paths to samples list and deseq2 results
samples_file = f'{repo_dir}/data/?/SRR_Acc_List.txt'
samples_organoid_file = f'{repo_dir}/data/?/SRR_Acc_List-organoid.txt'
deseq2_file = f'{data_dir}/?/deseq2/deseq2_result'
dge_gsea_dir = f'{results_path}/?'
for dataset in dat_met:
    dat_met[dataset]['samples_file']=samples_file.replace('?',dataset)
    dat_met[dataset]['samples_organoid_file']=samples_organoid_file.replace('?',dataset)
    dat_met[dataset]['deseq2_file']=deseq2_file.replace('?',dataset)
    dat_met[dataset]['dge_gsea_dir']=dge_gsea_dir.replace('?',dataset)
    # create datasets subfolders in results directory if do not exist
    os.makedirs(f'{results_path}/{dataset}',exist_ok=True)

In [ ]:
# import statistical test results: organoid -vs- tissue
colnames = ['GeneID',	'Base mean',	'log2(FC)',	'StdErr',	'Wald-Stats',	'P-value',	'P-adj']
# stuck results in one table
deseq2_data = []
for dataset in dat_met:
    results_path = dat_met[dataset]['deseq2_file']
    df = pd.read_csv(results_path, sep='\t', names = colnames)
    deseq2_data['dataset'] = dataset
    deseq2_data.append(df)
deseq2_data = pd.concat(deseq2_data, ignore_index=True)

In [ ]:
# create function that visualises deseq2 results in volcano plot
def dge_volcano_plot(data, save_dir, title=None, fc_thresh = 2, pval_thresh = 0.01):
    """
    Input:
        data - deseq2 result for one dataset
        save_dir - directory where save the plot
        title - plot name (optional)
        fc_thresh - Log2(Fold Change) threshold (default 2)
        pval_thresh - p-value threshold (default 0.01)
    """
    # Set significance threshold
    sig_thresh = -np.log10(pval_thresh)
    # Define colors
    data['color'] = 'grey'
    data.loc[(data['log2(FC)'] > fc_thresh) & (data['P-adj'] < pval_thresh), 'color'] = 'red'
    data.loc[(data['log2(FC)'] < -fc_thresh) & (data['P-adj'] < pval_thresh), 'color'] = 'blue'
    # Transform p-value
    data['-log10(P-adj)'] = -np.log(data['P-adj'])
    # Plot
    plt.figure(figsize=(8,6))
    plt.scatter(data['log2(FC)'], data['-log10(P-adj)'], c=data['color'], alpha=0.7)
    # Threshold lines
    plt.axvline(fc_thresh, color='black', linestyle='--')
    plt.axvline(-fc_thresh, color='black', linestyle='--')
    plt.axhline(sig_thresh, color='black', linestyle='--')
    # Axis annotation
    if title: plt.title(title)
    plt.xlabel('log2 Fold Change')
    plt.ylabel('-log10 p-value adjusted')
    plt.savefig(f"{save_dir}/volcanoplot.svg", format="svg", bbox_inches ='tight' )
    plt.show()

In [ ]:
# do volcano plot for each dataset
for dataset, data in deseq2_data.groupby('dataset'):
    save_dir = dat_met[dataset]['dge_gsea_dir']
    title = f"{dat_met[dataset]['name']}"
    dge_volcano_plot(data, save_dir, title)

In [5]:
# create function to do GSEA analysis
def gsea_dotplot(data, save_dir, title=None, gene_sets='KEGG_2021_Human', organism='Human', fdr = 0.05):
    """
    Input:
        data - deseq2 result for one dataset
        save_dir - directory where save the plot
        title - plot name (optional)
        gene_sets - catalog of gene sets (default KEGG_2021_Human)
        organism - (defult Human)
        fdr - corrected p-value threshold (default 0.05)
    """
    # rank genes
    rnk = data.sort_values('log2(FC)', ascending=False)[["GeneID", 'log2(FC)']]
    # GSEA
    gsea_results = gp.prerank(
        rnk=rnk,
        gene_sets=gene_sets,
        organism=organism,
        permutation_num=1000,
        seed=42,
        verbose=True
    )
    # convert into table
    res = gsea_results.res2d
    # filter significant pathways
    sig = res[res["fdr"] < fdr]
    # plot pathway gene sets
    ax = gp.dotplot(sig)
    if title: plt.title(title)
    plt.savefig(f"{save_dir}/{gene_sets}_dotplot.svg", format="svg", bbox_inches ='tight' )
    plt.close()